# Installing MASE (again)

Run the block below to install MASE in the current Colab runtime

In [ ]:
""" git_token = "YOUR_GIT_TOKEN"
short_code = "YOUR_SHORT_CODE"

# Check the current python version (It should be using Python 3.10) and update pip to the latest version.
!python --version
!python -m pip install --user --upgrade pip

# Clone MASE from your branch (the branch must already exist)
!git clone -b lab1_{short_code} https://{git_token}@github.com/DeepWok/mase.git

# Install requirements
!python -m pip install -r ./mase/machop/requirements.txt

# Change working directory to machop
%cd ./mase/machop/ """

' git_token = "YOUR_GIT_TOKEN"\nshort_code = "YOUR_SHORT_CODE"\n\n# Check the current python version (It should be using Python 3.10) and update pip to the latest version.\n!python --version\n!python -m pip install --user --upgrade pip\n\n# Clone MASE from your branch (the branch must already exist)\n!git clone -b lab1_{short_code} https://{git_token}@github.com/DeepWok/mase.git\n\n# Install requirements\n!python -m pip install -r ./mase/machop/requirements.txt\n\n# Change working directory to machop\n%cd ./mase/machop/ '

In [3]:
%pwd
%cd /home/rotask/mase/machop

/home/rotask/mase/machop


# General introduction

In this lab, you will learn how to use the search functionality in the software stack of MASE.

There are in total 4 tasks you would need to finish.

# Writing a search using MaseGraph Transforms

In this section, our objective is to gain a comprehensive understanding of the construction of the current search function in Mase. To achieve this, we will require these essential components:

- MaseGraph: This component should be already created in the preceding lab.
- Search space: This component encompasses and defines the various available search options.
- Search strategy: An implementation of a search algorithm.
- Runner: This vital component manages and executes training, evaluation, or both procedures while generating a quality metric.

By analyzing these components, we can delve into the workings and effectiveness of the existing search function in Mase.

# Turning your network to a graph

We follow a similar procedure of what you have tried in lab2 to now produce a MaseGraph, this is converted from your pre-trained JSC model:

In [2]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# # figure out the correct path
# machop_path = Path(".").resolve().parent.parent /"machop"
# assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
# sys.path.append(str(machop_path))

from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph.analysis import (
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
)
from chop.passes.graph import (
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.ir.graph.mase_graph import MaseGraph

from chop.models import get_model_info, get_model




set_logging_verbosity("info")

batch_size = 8
model_name = "jsc-tiny"
dataset_name = "jsc"


data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
    # custom_dataset_cache_path="../../chop/dataset"
)
data_module.prepare_data()
data_module.setup()

model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False,
    checkpoint = None)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

/home/rotask/anaconda3/envs/mase/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/rotask/anaconda3/envs/mase/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/rotask/anaconda3/envs/mase/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/rotask/anaconda3/envs/mase/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.r

#Defining a search space

Based on the previous `pass_args` template, the following code is utilized to generate a search space. The search space is constructed by combining different weight and data configurations in precision setups.

In [3]:
pass_args = {
"by": "type",
"default": {"config": {"name": None}},
"linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
},}

import copy
# build a search space
data_in_frac_widths = [(16, 8), (8, 6), (8, 4), (4, 2)]
w_in_frac_widths = [(16, 8), (8, 6), (8, 4), (4, 2)]
search_spaces = []
for d_config in data_in_frac_widths:
    for w_config in w_in_frac_widths:
        pass_args['linear']['config']['data_in_width'] = d_config[0]
        pass_args['linear']['config']['data_in_frac_width'] = d_config[1]
        pass_args['linear']['config']['weight_width'] = w_config[0]
        pass_args['linear']['config']['weight_frac_width'] = w_config[1]
        # dict.copy() and dict(dict) only perform shallow copies
        # in fact, only primitive data types in python are doing implicit copy when a = b happens
        search_spaces.append(copy.deepcopy(pass_args))

## Defining a search strategy and a runner

The code provided below consists of two main `for` loops. The first `for` loop executes a straightforward brute-force search, enabling the iteration through the previously defined search space.

In contrast, the second `for` loop retrieves training samples from the train data loader. These samples are then utilized to generate accuracy and loss values, which serve as potential quality metrics for evaluating the system's performance.


In [ ]:
# grid search
import torch
from torchmetrics.classification import MulticlassAccuracy

from chop.passes.graph.transforms import (
    quantize_transform_pass,
    summarize_quantization_analysis_pass,
)

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

metric = MulticlassAccuracy(num_classes=5)
num_batchs = 5
# This first loop is basically our search strategy,
# in this case, it is a simple brute force search

recorded_accs = []
for i, config in enumerate(search_spaces):
    mg, _ = quantize_transform_pass(mg, config)
    j = 0

    # this is the inner loop, where we also call it as a runner.
    acc_avg, loss_avg = 0, 0
    accs, losses = [], []
    for inputs in data_module.train_dataloader():
        xs, ys = inputs
        preds = mg.model(xs)
        loss = torch.nn.functional.cross_entropy(preds, ys)
        acc = metric(preds, ys)
        accs.append(acc)
        losses.append(loss)
        if j > num_batchs:
            break
        j += 1
    acc_avg = sum(accs) / len(accs)
    loss_avg = sum(losses) / len(losses)
    recorded_accs.append(acc_avg)

In [ ]:
print(recorded_accs)
print(acc_avg)
print(loss_avg)

[tensor(0.3619), tensor(0.2857), tensor(0.2948), tensor(0.3036), tensor(0.1986), tensor(0.2345), tensor(0.2155), tensor(0.3304), tensor(0.2637), tensor(0.2762), tensor(0.2363), tensor(0.3601), tensor(0.3060), tensor(0.2208), tensor(0.2839), tensor(0.2857)]
tensor(0.2857)
tensor(1.5731, grad_fn=<DivBackward0>)


In [ ]:
import time
import torch
import torch.nn.functional as F
from torchmetrics.classification import MulticlassAccuracy
from thop import profile
import copy

from chop.passes.graph.transforms import (
    quantize_transform_pass,
    summarize_quantization_analysis_pass,
)

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

metric = MulticlassAccuracy(num_classes=5)
num_batchs = 5
# This first loop is basically our search strategy,
# in this case, it is a simple brute force search

recorded_accs = []
recorded_latencies = []
recorded_sizes = []
flops_list = []

search_spaces.append(copy.deepcopy(pass_args))

for i, config in enumerate(search_spaces):
    

    # Store a copy of the original config
    original_config = copy.deepcopy(config)

    mg, _ = quantize_transform_pass(mg, config)
    j = 0

    acc_avg, loss_avg = 0, 0
    accs, losses = [], []
    latencies, sizes, flops = [], [], []
    for inputs in data_module.train_dataloader():
        xs, ys = inputs

        # Measure latency
        start_time = time.time()
        preds = mg.model(xs)
        end_time = time.time()
        latency = end_time - start_time
        latencies.append(latency)

        # Measure model size
        model_size = sum(p.element_size() * p.nelement() for p in mg.model.parameters())
        sizes.append(model_size)

        # Measure FLOPs
        flops, _ = profile(mg.model, inputs=(xs,))
        flops_list.append(flops)

        loss = F.cross_entropy(preds, ys)
        acc = metric(preds, ys)
        accs.append(acc)
        losses.append(loss)
        if j > num_batchs:
            break
        j += 1

    acc_avg = sum(accs) / len(accs)
    loss_avg = sum(losses) / len(losses)
    latency_avg = sum(latencies) / len(latencies)
    size_avg = sum(sizes) / len(sizes)
    flops_avg = sum(flops_list) / len(flops_list)

    recorded_accs.append(acc_avg)
    recorded_latencies.append(latency_avg)
    recorded_sizes.append(size_avg)
    flops_list.append(flops_avg)

AttributeError: 'GraphModule' object has no attribute 'additional_inputs'

In [ ]:
print(flops_avg)
print(recorded_accs)
print(latency_avg)
print(size_avg)

512.0
[tensor(0.2405), tensor(0.1958), tensor(0.2607), tensor(0.1750), tensor(0.1756), tensor(0.2083), tensor(0.2024), tensor(0.3893), tensor(0.2452), tensor(0.1583), tensor(0.1536), tensor(0.2333), tensor(0.1714), tensor(0.1964), tensor(0.1726), tensor(0.2310)]
0.00046334947858537944
468.0


In [ ]:
# Reinstate the original config
config = copy.deepcopy(original_config)

In [ ]:
search_spaces

[{'default': {'config': {'name': None}},
  'linear': {'config': {'name': 'integer',
    'data_in_width': 16,
    'data_in_frac_width': 8,
    'weight_width': 16,
    'weight_frac_width': 8,
    'bias_width': 8,
    'bias_frac_width': 4}}},
 {'default': {'config': {'name': None}},
  'linear': {'config': {'name': 'integer',
    'data_in_width': 16,
    'data_in_frac_width': 8,
    'weight_width': 8,
    'weight_frac_width': 6,
    'bias_width': 8,
    'bias_frac_width': 4}}},
 {'default': {'config': {'name': None}},
  'linear': {'config': {'name': 'integer',
    'data_in_width': 16,
    'data_in_frac_width': 8,
    'weight_width': 8,
    'weight_frac_width': 4,
    'bias_width': 8,
    'bias_frac_width': 4}}},
 {'default': {'config': {'name': None}},
  'linear': {'config': {'name': 'integer',
    'data_in_width': 16,
    'data_in_frac_width': 8,
    'weight_width': 4,
    'weight_frac_width': 2,
    'bias_width': 8,
    'bias_frac_width': 4}}},
 {'default': {'config': {'name': None}},
  

In [4]:
import time
import torch
import torch.nn.functional as F
from torchmetrics.classification import MulticlassAccuracy
from thop import profile
import copy

# Assuming all the necessary imports and initializations are done above
from chop.passes.graph.transforms import (
    quantize_transform_pass,
    summarize_quantization_analysis_pass,
)

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

metric = MulticlassAccuracy(num_classes=5)
num_batchs = 5

recorded_metrics = []

for i, config in enumerate(search_spaces):
    # Store a copy of the original config
    original_config = copy.deepcopy(config)

    mg, _ = quantize_transform_pass(mg, config)
    
    accs, losses = [], []
    latencies, sizes, flops_list = [], [], []
    
    for j, inputs in enumerate(data_module.train_dataloader()):
        xs, ys = inputs

        # Measure latency
        start_time = time.time()
        preds = mg.model(xs)
        end_time = time.time()
        latencies.append(end_time - start_time)

        # Measure model size
        sizes.append(sum(p.numel() * p.element_size() for p in mg.model.parameters() if p.requires_grad))

        # Measure FLOPs
        # Ensure xs is a single input batch and not the entire dataset
        flops, _ = profile(mg.model, inputs=(xs, ), verbose=False)
        flops_list.append(flops)

        # Calculate accuracy and loss
        loss = F.cross_entropy(preds, ys)
        acc = metric(preds, ys)
        accs.append(acc.item())
        losses.append(loss.item())

        if j >= num_batchs - 1:
            break

    # Calculate the averages
    acc_avg = sum(accs) / len(accs)
    loss_avg = sum(losses) / len(losses)
    latency_avg = sum(latencies) / len(latencies)
    size_avg = sum(sizes) / len(sizes)
    flops_avg = sum(flops_list) / len(flops_list)

    # Record the metrics 
    #bytes (model size)
    recorded_metrics.append({
        'accuracy': acc_avg,
        'loss': loss_avg,
        'latency': latency_avg,
        'model_size': size_avg,
        'flops': flops_avg,
        'config': original_config
    })


In [10]:
for metric in recorded_metrics:
        metric.pop('config', None)
        print(metric)

{'accuracy': 0.24833334013819694, 'loss': 1.6010365724563598, 'latency': 0.0013057708740234375, 'model_size': 468.0, 'flops': 512.0}
{'accuracy': 0.16666667014360428, 'loss': 1.703313660621643, 'latency': 0.0007787704467773438, 'model_size': 468.0, 'flops': 512.0}
{'accuracy': 0.2066666752099991, 'loss': 1.644864797592163, 'latency': 0.0009151458740234375, 'model_size': 468.0, 'flops': 512.0}
{'accuracy': 0.17333333790302277, 'loss': 1.6393198490142822, 'latency': 0.0010591506958007812, 'model_size': 468.0, 'flops': 512.0}
{'accuracy': 0.22666666954755782, 'loss': 1.647191834449768, 'latency': 0.0007370948791503906, 'model_size': 468.0, 'flops': 512.0}
{'accuracy': 0.18666667342185975, 'loss': 1.6225704669952392, 'latency': 0.0009089946746826172, 'model_size': 468.0, 'flops': 512.0}
{'accuracy': 0.18333333879709243, 'loss': 1.65449116230011, 'latency': 0.001065540313720703, 'model_size': 468.0, 'flops': 512.0}
{'accuracy': 0.24000000655651094, 'loss': 1.618965458869934, 'latency': 0.00

We now have the following task for you:

1. Explore additional metrics that can serve as quality metrics for the search process. For example, you can consider metrics such as latency, model size, or the number of FLOPs (floating-point operations) involved in the model.

2. Implement some of these additional metrics and attempt to combine them with the accuracy or loss quality metric. It's important to note that in this particular case, accuracy and loss actually serve as the same quality metric (do you know why?).



# The search command in the MASE flow

The search flow implemented in MASE is very similar to the one that you have constructed manually, the overall flow is implemented in [search.py](../../machop/chop/actions/search/search.py), the following bullet points provide you pointers to the code base.

- MaseGraph: this is the [MaseGraph](../../machop/chop/passes/graph/mase_graph.py) that you have used in lab2.
- Search space: The base class is implemented in [base.py](../../machop/chop/actions/search/search_space/base.py) , where in the same folder you can see a range of different supported search spaces.
- Search strategy: Similar to the search space, you can find a a base class [definition](../../machop/chop/actions/search/strategies/base.py), where different strategies are also defined in the same folder.
- Runner: Different [runners](../../machop/chop/actions/search/strategies/runners) can produce different metrics, they may also use `transforms` to help compute certain search metrics.

This enables one to execute the search through the MASE command line interface, remember to change the name after the `--load` option.


In [4]:
!pwd

/home/rotask/mase/machop


In [8]:
!./ch search --config configs/examples/jsc_toy_by_type.toml --load ../mase_output/jsc-tiny_classification_jsc_2024-01-26/software/training_ckpts/best.ckpt --load-type pl

/home/rotask/anaconda3/envs/mase/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/rotask/anaconda3/envs/mase/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/rotask/anaconda3/envs/mase/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/rotask/anaconda3/envs/mase/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.r

In this scenario, the search functionality is specified in the `toml` configuration file rather than via command-line inputs. This approach is adopted due to the multitude of configuration parameters that need to be set; encapsulating them within a single, elegant configuration file enhances reproducibility.

In `jsc_toy_by_type.toml`, the `search_space` configuration is set in `search.search_space`, the search strategy is configured via `search.strategy`. If you are not familiar with the `toml` syntax, you can read [here](https://toml.io/en/v1.0.0).

> In order to accomplish the following task, it is necessary to make direct modifications to the code base. This can be challenging within the Colab environment. **It is recommended to implement the task on a local setup and utilize Colab strictly as a server to execute the search command above.** Consider Colab as a dedicated server for this purpose.

With now an understanding of how the MASE flow work, consider the following tasks

3. Implement the brute-force search as an additional search method within the system, this would be a new search strategy in MASE.
4. Compare the brute-force search with the TPE based search, in terms of sample efficiency. Comment on the performance difference between the two search methods.